In [1]:
import pandas as pd
import glob
import os
import sys
from tqdm import tqdm

In [2]:
# KPI Data
path = "C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Host\\"
all_files = glob.glob(path + "*.csv")
all_files

['C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Host\\db_oracle_11g.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Host\\dcos_container.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Host\\dcos_docker.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Host\\mw_redis.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Host\\os_linux.csv']

In [35]:
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    dfs.append(df)
df = pd.concat(dfs, axis=0, ignore_index=True)
df.head()

In [13]:
df = df.sort_values(["timestamp"], ignore_index=True)
df = df[(df.timestamp >= (1590441300000-1200000)) & (df.timestamp <= (1590441300000 + 300000))]

In [15]:
df.to_csv('kpi_data_526.csv', index=False)

In [14]:
df.head()

,itemid,name,bomc_id,timestamp,value,cmdb_id
488184,999999996430680,CPU_util_pct,ZJ-001-001,1590440100000,0.112063,os_010
488185,999999996437700,CPU_util_pct,ZJ-001-001,1590440100000,1.620712,os_002
488186,65435640,Disk_io_util,ZJ-001-125,1590440100000,0.000000,os_020
488187,999999996432300,Num_of_processes,ZJ-001-076,1590440100000,224.000000,os_009
488188,999999996432240,Disk_rd_ios,ZJ-001-098,1590440100000,0.000000,os_007


In [2]:
path = "C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\"
all_files = glob.glob(path + "*.csv")
all_files

['C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_csf.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_fly_remote.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_jdbc.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_local.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_osb.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_05_26\\Trace\\trace_remote_process.csv']

In [3]:
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    dfs.append(df)

In [4]:
for df in dfs:
    if df['callType'].iloc[0]=='JDBC':
        df['serviceName'] = df['dsName']
    # else:
    #     df['serviceName'] = df['cmdb_id']
    # if df['callType'].iloc[0]=='JDBC':
    #     df['serviceName'] = df['dsName']

In [5]:
df = pd.concat(dfs, axis=0, ignore_index=True)
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName,dsName
0,CSF,1590422400042,3212.0,True,fe171a0d9608272247dd,fe1718e7207827a247dd,fe171ae35818272247dd,os_021,csf_001,NaN
1,CSF,1590422400043,521.0,True,fe171b1dc723053247dd,fe1712efc05305a247dd,fe171260ff33053247dd,os_022,csf_001,NaN
2,CSF,1590422400043,2483.0,True,fe1710044423683247dd,fe171c71bc7368b247dd,fe1711b3b433683247dd,os_021,csf_001,NaN
3,CSF,1590422400044,2230.0,True,fe1717f21d28273247dd,fe1712d8e85827c247dd,fe17174eec38273247dd,os_021,csf_001,NaN
4,CSF,1590422400044,2482.0,True,fe171972b143684247dd,fe171f20ea9368c247dd,fe1719218f53684247dd,os_021,csf_001,NaN


In [6]:
df = df.drop('dsName', axis=1)
# del dfs

In [7]:
df = df[(df.startTime >= (1590441300000-1200000)) & (df.startTime <= (1590441300000 + 300000))]

In [8]:
df.serviceName.unique()

array(['os_022', 'os_021', 'docker_003', 'docker_004', 'docker_002',
       'docker_001', 'db_003', 'db_009', 'db_007', 'docker_005',
       'docker_006', 'docker_008', 'docker_007'], dtype=object)

In [8]:
df.cmdb_id.unique()

array(['os_022', 'os_021', 'docker_001', 'docker_002', 'docker_003',
       'docker_004', 'docker_008', 'docker_007', 'docker_005',
       'docker_006'], dtype=object)

In [8]:
df = df.sort_values(["startTime"], ignore_index=True)
# # df = df.sort_values(["traceId", "startTime"], ignore_index=True)
# df = df[df.startTime < df.startTime[0]+600000]
# df

In [9]:
df1 = df[df['callType']=='RemoteProcess']
df1 = df1[['pid','cmdb_id']]
df1 = df1.set_index('pid')

In [10]:
csf_dict = df1.to_dict()
csf_cmdb = {str(key):str(values) for key, values in csf_dict['cmdb_id'].items()}

In [11]:
for index, row in tqdm(df.iterrows()):
    if row['id'] in csf_cmdb:
        df.at[index, 'cmdb_id'] = csf_cmdb[row['id']]

690099it [00:49, 13999.15it/s]


In [27]:
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
0,OSB,1590440100022,443.0,True,0f1716d7b664266b88be,0f171c2e2374266b88be,None,os_021,os_021
1,OSB,1590440100024,537.0,True,0f171040c760528b88be,0f17176e2e70528b88be,None,os_022,os_022
2,OSB,1590440100026,767.0,True,0f171f3ed80526ab88be,0f171bc4e51526ab88be,None,os_021,os_021
3,OSB,1590440100027,1678.0,True,0f171278d90526bb88be,0f171b2cdb1526bb88be,None,os_021,os_021
4,OSB,1590440100027,1660.0,True,0f171c055d7052bb88be,0f171d035a8052bb88be,None,os_022,os_022


In [12]:
elapse_time = {}
children = {}
parent_service = {}
for index, row in tqdm(df.iterrows()):
    if row['pid'] != 'None':
        if row['pid'] in children.keys():
            children[row['pid']].append(row['id'])
        else:
            children[row['pid']] = [row['id']]
    elapse_time [row['id']] = float(row['elapsedTime'])
    # parent_service[row['id']] = str(row['cmdb_id'] + ':' + row['serviceName'])
    parent_service[row['id']] = row['serviceName']


690099it [01:04, 10692.70it/s]


In [16]:
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
0,OSB,1587579090784,1540.0,True,00007171a31677604200,b27e5171a31677604201,None,os_022,osb_001
1,CSF,1587579090790,1525.0,True,00007171a31677604200,acdcd171a31677664203,b27e5171a31677604201,os_022,docker_002
2,RemoteProcess,1587579090796,1520.0,True,00007171a31677604200,2e8ba171a316776c9828,acdcd171a31677664203,docker_002,csf_001
3,FlyRemote,1587579090997,5.0,True,00007171a31677604200,23a11171a31678359892,2e8ba171a316776c9828,docker_002,fly_remote_001
4,CSF,1587579091097,347.0,True,00007171a31677604200,76499171a31678989929,2e8ba171a316776c9828,docker_002,docker_007


In [13]:
df['actual_time'] = 0.0
df['path'] = ''
for index, row in tqdm(df.iterrows()):
    total_child = 0.0
    if row['pid'] not in parent_service.keys():
        df.at[index, 'path'] = 'Start-' + row['serviceName']
    else:
        df.at[index, 'path'] = parent_service[row['pid']] + '-' + row['serviceName']

    if row['id'] not in children.keys():
        df.at[index, 'actual_time'] = row['elapsedTime']
        continue
    for child in children[row['id']]:
        total_child += elapse_time[child]
    df.at[index, 'actual_time'] = row['elapsedTime'] - total_child

690099it [01:15, 9165.78it/s]


In [14]:
df = df.sort_values(["startTime"], ignore_index=True)

In [18]:
df.head(20)

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName,actual_time,path
0,OSB,1590440100022,443.0,True,0f1716d7b664266b88be,0f171c2e2374266b88be,None,os_021,osb_001,20.0,Start-osb_001
1,OSB,1590440100024,537.0,True,0f171040c760528b88be,0f17176e2e70528b88be,None,os_022,osb_001,19.0,Start-osb_001
2,OSB,1590440100026,767.0,True,0f171f3ed80526ab88be,0f171bc4e51526ab88be,None,os_021,osb_001,31.0,Start-osb_001
3,OSB,1590440100027,1678.0,True,0f171278d90526bb88be,0f171b2cdb1526bb88be,None,os_021,osb_001,19.0,Start-osb_001
4,OSB,1590440100027,1660.0,True,0f171c055d7052bb88be,0f171d035a8052bb88be,None,os_022,osb_001,17.0,Start-osb_001
5,OSB,1590440100030,539.0,True,0f171dbef10152eb88be,0f1715b8421152eb88be,None,os_022,osb_001,16.0,Start-osb_001
6,CSF,1590440100030,423.0,True,0f1716d7b664266b88be,0f171ec2a89426eb88be,0f171c2e2374266b88be,docker_003,csf_001,6.0,osb_001-csf_001
7,CSF,1590440100031,518.0,True,0f171040c760528b88be,0f1716969d3152fb88be,0f17176e2e70528b88be,docker_001,csf_001,8.0,osb_001-csf_001
8,RemoteProcess,1590440100032,510.0,True,0f171040c760528b88be,0f1711a8a558520c88be,0f1716969d3152fb88be,docker_001,csf_001,39.0,csf_001-csf_001
9,RemoteProcess,1590440100033,400.0,True,0f171c055d7052bb88be,0f17151c1f78521c88be,0f171f7d0911521c88be,docker_001,csf_001,107.0,csf_001-csf_001


In [19]:
df.to_csv('trace_untouched_5_26.csv', index=False)

In [21]:
df.serviceName.unique()

array(['osb_001', 'docker_002', 'csf_001', 'fly_remote_001', 'docker_007',
       'csf_002', 'db_003', 'csf_003', 'docker_008', 'csf_004', 'csf_005',
       'db_009', 'db_007', 'docker_004', 'docker_006', 'docker_005',
       'docker_003', 'docker_001'], dtype=object)

In [22]:
df.cmdb_id.unique()

array(['os_022', 'docker_002', 'docker_007', 'docker_008', 'os_021',
       'docker_004', 'docker_006', 'docker_005', 'docker_003',
       'docker_001'], dtype=object)

In [23]:
df.head(60)

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName,actual_time,path
0,OSB,1587579090784,1540.0,True,00007171a31677604200,b27e5171a31677604201,None,os_022,osb_001,15.0,Start-osb_001
1,CSF,1587579090790,1525.0,True,00007171a31677604200,acdcd171a31677664203,b27e5171a31677604201,os_022,docker_002,5.0,osb_001-docker_002
2,RemoteProcess,1587579090796,1520.0,True,00007171a31677604200,2e8ba171a316776c9828,acdcd171a31677664203,docker_002,csf_001,569.0,docker_002-csf_001
3,FlyRemote,1587579090997,5.0,True,00007171a31677604200,23a11171a31678359892,2e8ba171a316776c9828,docker_002,fly_remote_001,5.0,csf_001-fly_remote_001
4,CSF,1587579091097,347.0,True,00007171a31677604200,76499171a31678989929,2e8ba171a316776c9828,docker_002,docker_007,297.0,csf_001-docker_007
5,RemoteProcess,1587579091394,50.0,True,00007171a31677604200,84370171a31679c28884,76499171a31678989929,docker_007,csf_002,33.0,docker_007-csf_002
6,JDBC,1587579091412,8.0,True,00007171a31677604200,a5388171a31679d48894,a41f6171a31679d48891,docker_007,db_003,8.0,db_003-db_003
7,LOCAL,1587579091412,17.0,True,00007171a31677604200,a41f6171a31679d48891,84370171a31679c28884,docker_007,db_003,2.0,csf_002-db_003
8,JDBC,1587579091421,7.0,True,00007171a31677604200,eba32171a31679dd8897,a41f6171a31679d48891,docker_007,db_003,7.0,db_003-db_003
9,CSF,1587579091596,300.0,True,00007171a31677604200,7a21a171a3167a249974,2e8ba171a316776c9828,docker_002,docker_007,253.0,csf_001-docker_007
